<a href="https://colab.research.google.com/github/ankak2635/Credit-Card-Fraud-Detection/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing the dataset using Kaggle API ⤵📄**

Credit card dataset from kaggle using API

In [ ]:
#installing the Kaggle library
!pip install kaggle

In [ ]:
# configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#API to fetch dataset from kaggle
!kaggle datasets download -d mlg-ulb/creditcardfraud

 82% 54.0M/66.0M [00:00<00:00, 73.4MB/s]
100% 66.0M/66.0M [00:00<00:00, 122MB/s] 


In [ ]:
# Extracting the zip files
from zipfile import ZipFile
dataset_path = '/content/creditcardfraud.zip'

with ZipFile(dataset_path, 'r') as zip:
  zip.extractall()
  print("The dataset has been extracted")

The dataset has been extracted


## **Credit Card Fraud Detection Project**

Importing the libraries.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# loading the dataset
credit_card_df = pd.read_csv('/content/creditcard.csv')

**EDA**

In [ ]:
credit_card_df.shape

(284807, 31)

In [ ]:
credit_card_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


The variables names are not descriptive due to privacy reasons

In [ ]:
credit_card_df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,-0.915427,-1.040458,-0.031513,-0.188093,-0.084316,0.041333,-0.302620,-0.660377,0.167430,-0.256117,0.382948,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


Class 1 --> Fraud

Class 0 ---> Legit

In [ ]:
credit_card_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
# check for missing values
credit_card_df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [ ]:
# stats insight
credit_card_df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,2.040130e-15,-1.698953e-15,-1.893285e-16,-3.147640e-15,1.772925e-15,9.289524e-16,-1.803266e-15,1.674888e-15,1.475621e-15,3.501098e-15,1.392460e-15,-7.466538e-16,4.258754e-16,9.019919e-16,5.126845e-16,1.473120e-16,8.042109e-16,5.282512e-16,4.456271e-15,1.426896e-15,1.701640e-15,-3.662252e-16,-1.217809e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,1.088850e+00,1.020713e+00,9.992014e-01,9.952742e-01,9.585956e-01,9.153160e-01,8.762529e-01,8.493371e-01,8.381762e-01,8.140405e-01,7.709250e-01,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,-2.458826e+01,-4.797473e+00,-1.868371e+01,-5.791881e+00,-1.921433e+01,-4.498945e+00,-1.412985e+01,-2.516280e+01,-9.498746e+00,-7.213527e+00,-5.449772e+01,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,-5.354257e-01,-7.624942e-01,-4.055715e-01,-6.485393e-01,-4.255740e-01,-5.828843e-01,-4.680368e-01,-4.837483e-01,-4.988498e-01,-4.562989e-01,-2.117214e-01,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,-9.291738e-02,-3.275735e-02,1.400326e-01,-1.356806e-02,5.060132e-02,4.807155e-02,6.641332e-02,-6.567575e-02,-3.636312e-03,3.734823e-03,-6.248109e-02,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,4.539234e-01,7.395934e-01,6.182380e-01,6.625050e-01,4.931498e-01,6.488208e-01,5.232963e-01,3.996750e-01,5.008067e-01,4.589494e-01,1.330408e-01,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,2.374514e+01,1.201891e+01,7.848392e+00,7.126883e+00,1.052677e+01,8.877742e+00,1.731511e+01,9.253526e+00,5.041069e+00,5.591971e+00,3.942090e+01,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [ ]:
# distribution of legit and fraud transaction
credit_card_df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

The dataset is highly unbalanced.

In [ ]:
credit_card_df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [ ]:
# separating the data for analysis
legit = credit_card_df[credit_card_df.Class == 0]
fraud = credit_card_df[credit_card_df.Class == 1]

In [ ]:
legit.shape

(284315, 31)

In [ ]:
fraud.shape

(492, 31)

In [ ]:
legit['Amount'].describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [ ]:
fraud['Amount'].describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

# **Treating the disbalance**

**Under-Sampling**

Create a df which contains similar amount of both classes' observations

In [ ]:
legit_sample = legit.sample(n=492)

In [ ]:
# Concat legit_sample and fraud dataframes
final_df = pd.concat([legit_sample, fraud], axis = 0)

In [ ]:
final_df.shape

(984, 31)

In [ ]:
final_df['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [ ]:
final_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
224878,144008.0,1.439274,-1.297654,-0.972792,0.365315,-0.493893,0.313115,-0.265567,0.170182,1.101828,-0.099759,0.414414,0.494655,-0.977897,0.462744,0.207315,0.526263,-0.814546,0.484634,0.256814,0.311261,-0.045160,-0.739521,0.144515,0.222277,-0.551963,-0.713591,-0.035252,0.006690,284.06,0
24580,33286.0,1.156804,-1.061558,0.155120,-0.391178,-1.124593,-0.442674,-0.536152,0.022241,-0.199850,0.480399,-0.759298,-1.434965,-2.175369,0.076057,-0.063408,0.292701,1.126782,-1.914939,0.558862,0.083121,0.118652,0.141891,-0.140772,0.063008,0.501518,-0.119750,-0.011595,0.018584,114.71,0
134915,81029.0,0.935829,-0.889857,0.837171,0.158804,-1.141901,0.192212,-0.628429,0.227231,1.052484,-0.326479,0.454477,0.805575,-0.653533,-0.225243,-0.888652,0.235762,-0.205631,0.086922,0.861537,0.178625,-0.133450,-0.493871,-0.090177,0.022961,0.086833,0.936350,-0.067564,0.022352,142.76,0
82139,59260.0,-1.433146,0.639578,1.662322,0.995898,-0.267489,1.276883,0.040332,0.641845,0.101903,-0.078960,1.090883,1.219297,-0.004132,-0.194835,-0.572169,-1.126183,0.649586,-0.467393,0.402686,-0.029056,0.107777,0.787445,-0.225106,-0.228515,-0.057828,-0.211588,-0.044446,-0.148680,77.90,0
31390,36346.0,-0.634011,0.555985,0.829155,-1.747728,0.279131,0.792218,-0.826181,-2.333646,-0.365675,-1.518230,0.246987,0.300809,-1.209204,0.894066,-0.183687,1.478731,-1.190079,-0.218934,-1.452054,0.459371,-1.621988,-0.976576,-0.180052,-1.199925,1.148211,0.411945,-0.120555,0.128903,40.00,0


In [ ]:
final_df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,91455.367886,0.051932,-0.031319,0.031715,-0.040062,0.026226,-0.019987,0.067010,-0.029009,-0.027295,0.009338,0.008563,-0.084317,-0.009407,-0.007449,0.016423,0.052077,0.030927,-0.002089,0.021199,-0.006889,0.015847,-0.007671,-0.060241,-0.02178,0.009958,0.009236,-0.013527,0.011017,98.022358
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.10513,0.041449,0.051648,0.170575,0.075667,122.211321


In [ ]:
credit_card_df.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


The sample represents the original data fairly good.

# **Preprocessing & Model Training**

Separating target from features

In [ ]:
X = final_df.drop('Class', axis = 1)
Y = final_df['Class']

In [ ]:
X.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
224878,144008.0,1.439274,-1.297654,-0.972792,0.365315,-0.493893,0.313115,-0.265567,0.170182,1.101828,-0.099759,0.414414,0.494655,-0.977897,0.462744,0.207315,0.526263,-0.814546,0.484634,0.256814,0.311261,-0.045160,-0.739521,0.144515,0.222277,-0.551963,-0.713591,-0.035252,0.006690,284.06
24580,33286.0,1.156804,-1.061558,0.155120,-0.391178,-1.124593,-0.442674,-0.536152,0.022241,-0.199850,0.480399,-0.759298,-1.434965,-2.175369,0.076057,-0.063408,0.292701,1.126782,-1.914939,0.558862,0.083121,0.118652,0.141891,-0.140772,0.063008,0.501518,-0.119750,-0.011595,0.018584,114.71
134915,81029.0,0.935829,-0.889857,0.837171,0.158804,-1.141901,0.192212,-0.628429,0.227231,1.052484,-0.326479,0.454477,0.805575,-0.653533,-0.225243,-0.888652,0.235762,-0.205631,0.086922,0.861537,0.178625,-0.133450,-0.493871,-0.090177,0.022961,0.086833,0.936350,-0.067564,0.022352,142.76
82139,59260.0,-1.433146,0.639578,1.662322,0.995898,-0.267489,1.276883,0.040332,0.641845,0.101903,-0.078960,1.090883,1.219297,-0.004132,-0.194835,-0.572169,-1.126183,0.649586,-0.467393,0.402686,-0.029056,0.107777,0.787445,-0.225106,-0.228515,-0.057828,-0.211588,-0.044446,-0.148680,77.90
31390,36346.0,-0.634011,0.555985,0.829155,-1.747728,0.279131,0.792218,-0.826181,-2.333646,-0.365675,-1.518230,0.246987,0.300809,-1.209204,0.894066,-0.183687,1.478731,-1.190079,-0.218934,-1.452054,0.459371,-1.621988,-0.976576,-0.180052,-1.199925,1.148211,0.411945,-0.120555,0.128903,40.00


In [ ]:
Y.head()

224878    0
24580     0
134915    0
82139     0
31390     0
Name: Class, dtype: int64

**Train test split**

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 1, stratify = Y)

Check the split ratio

In [ ]:
print(Counter(Y_train), Counter(Y_test))

Counter({0: 394, 1: 393}) Counter({1: 99, 0: 98})


Both train and test data contains 50:50 of 0s and 1s

**Model Training**

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

**Model Evaluation**

In [ ]:
# Accuracy score on training data
train_pred = model.predict(X_train)
acc_train = accuracy_score(train_pred, Y_train)
print("Accuracy score on trianing data:",acc_train)

Accuracy score on trianing data: 0.9491740787801779


In [ ]:
# Accuracy score on test data
test_pred = model.predict(X_test)
acc_test = accuracy_score(test_pred, Y_test)
print("Accuracy score on test data:",acc_test)

Accuracy score on test data: 0.9137055837563451


**Model does not suffer from overfitting**

# **Building a predictive system**

First input data are the parameters of a legit transaction

Second input data are the parameters of a fraudulent transcation

In [ ]:
# input_data = (98967,0.0367970486610003,0.805747796467771,0.222735338586993,-0.59456499468351,0.625669526327528,-0.510805352830764,0.714699120294125,-0.0258757724098354,1.2408518390186,-0.569893656528747,1.03640225900994,-2.64139340581378,0.139589673002314,2.01536165553833,-1.64085551204399,0.284103109599178,-0.0140906798756155,0.361483174449366,0.191436576492114,-0.148921559719423,-0.407166727878275,-0.856256047708593,0.0322221962381484,-0.602162865600195,-0.529386746305225,0.11673956948849,0.205585633032438,0.0728095159488246,5.25)
input_data = (472.0, -3.0435406239976, -3.15730712090228, 1.08846277997285, 2.2886436183814, 1.35980512966107, -1.06482252298131, 0.325574266158614, -0.0677936531906277, -0.270952836226548, -0.838586564582682, -0.414575448285725, -0.503140859566824, 0.676501544635863, -1.69202893305906, 2.00063483909015, 0.666779695901966, 0.599717413841732, 1.72532100745514, 0.283344830149495, 2.10233879259444, 0.661695924845707, 0.435477208966341, 1.37596574254306, -0.293803152734021, 0.279798031841214, -0.145361714815161, -0.252773122530705, 0.0357642251788156, 529.0)
input_array = np.asarray(input_data)
input_array_reshaped = input_array.reshape(1,-1)
prediction = model.predict(input_array_reshaped)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
if (prediction[0] == 0):
  print('The credit card transaction is legit')

else:
  print('The credit card transaction is fraudulent')

The credit card transaction is fraudulent


Getting the parameters of a fraudulent transaction

In [ ]:
credit_card_df[credit_card_df['Class'] == 1].index

Int64Index([   541,    623,   4920,   6108,   6329,   6331,   6334,   6336,
              6338,   6427,
            ...
            274382, 274475, 275992, 276071, 276864, 279863, 280143, 280149,
            281144, 281674],
           dtype='int64', length=492)

In [ ]:
print(credit_card_df.iloc[623, :].tolist())

[472.0, -3.0435406239976, -3.15730712090228, 1.08846277997285, 2.2886436183814, 1.35980512966107, -1.06482252298131, 0.325574266158614, -0.0677936531906277, -0.270952836226548, -0.838586564582682, -0.414575448285725, -0.503140859566824, 0.676501544635863, -1.69202893305906, 2.00063483909015, 0.666779695901966, 0.599717413841732, 1.72532100745514, 0.283344830149495, 2.10233879259444, 0.661695924845707, 0.435477208966341, 1.37596574254306, -0.293803152734021, 0.279798031841214, -0.145361714815161, -0.252773122530705, 0.0357642251788156, 529.0, 1.0]
